# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data_HL/cities_HL.csv')
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6333,147.9167,-24.20,80,0,8.48,RU,1616138774
1,1,arraial do cabo,-22.9661,-42.0278,75.58,88,4,16.08,BR,1616138474
2,3,mar del plata,-38.0023,-57.5575,54.00,100,0,3.44,AR,1616138774
3,4,carnarvon,-24.8667,113.6333,91.40,33,0,13.80,AU,1616138775
4,5,sabha,27.0377,14.4283,73.31,16,0,14.54,LY,1616138775
...,...,...,...,...,...,...,...,...,...,...
546,593,vilyuysk,63.7553,121.6247,22.62,90,100,10.85,RU,1616139008
547,595,ubon ratchathani,15.1667,105.1667,100.00,49,0,5.75,TH,1616139009
548,596,ampanihy,-24.7000,44.7500,91.58,39,0,6.58,MG,1616139009
549,597,aksu,41.1231,80.2644,49.39,28,26,7.96,CN,1616139009


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)
locations = df[['Lat', 'Lng']]

figure_layout = {
    'width': '960px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'}

fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=df.Humidity.astype(float),
                                dissipating=False, max_intensity=df.Humidity.astype(float).max(),
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='960px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# filter max temp higher than 70
temp70 = df[df['Max Temp']>70]
# filter max temp lower than or equal to 80
temp = temp70[temp70['Max Temp']<=80]
# filter wind speed less than 10
wind = temp[temp['Wind Speed']<10]
# filter for zero cloudiness, get ideal weather dataframe
ideal = wind[wind.Cloudiness == 0]
ideal

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,98,richards bay,-28.7830,32.0377,79.72,71,0,3.38,ZA,1616138651
156,167,vila velha,-20.3297,-40.2925,79.00,94,0,4.61,BR,1616138840
157,168,puerto madero,14.7208,-92.4215,78.93,75,0,3.56,MX,1616138840
352,385,new norfolk,-42.7826,147.0587,73.00,49,0,1.99,AU,1616138925
371,404,sao filipe,14.8961,-24.4956,70.59,74,0,6.31,CV,1616138932
378,412,pochutla,15.7432,-96.4661,72.00,66,0,2.64,MX,1616138935
387,423,faya,18.3851,42.4509,73.40,21,0,8.05,SA,1616138939
434,472,boa vista,2.8197,-60.6733,77.00,73,0,5.75,BR,1616138960


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = pd.DataFrame(ideal, columns=['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
89,richards bay,ZA,-28.7830,32.0377,
156,vila velha,BR,-20.3297,-40.2925,
157,puerto madero,MX,14.7208,-92.4215,
352,new norfolk,AU,-42.7826,147.0587,
371,sao filipe,CV,14.8961,-24.4956,
378,pochutla,MX,15.7432,-96.4661,
387,faya,SA,18.3851,42.4509,
434,boa vista,BR,2.8197,-60.6733,


In [114]:
# the following block of code was used to see a sample API response before looping for each city later
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": "-28.7830,32.0377",
#     "type": "hotel",
#     "key": g_key,
#     "radius": 5000}
# response = requests.get(base_url, params=params).json()
# pprint(response['results'][1])

In [113]:
# find the first hotel for each city located within 5000 m of coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",
    "key": g_key,
    "radius": 5000}

for index, row in hotel_df.iterrows():
    params["location"] = f"{row.Lat},{row.Lng}"
    print(f"Retrieving Hotel for City: {row.City}")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {row.City} is {results[1]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[1]['name']
    # if no nearby hotel
    except (KeyError, IndexError):
        print("No hotels nearby.")
    print("------------")

Retrieving Hotel for City: richards bay
Closest hotel in richards bay is BON Hotel Waterfront Richards Bay.
------------
Retrieving Hotel for City: vila velha
Closest hotel in vila velha is Quality Suites Vila Velha.
------------
Retrieving Hotel for City: puerto madero
Closest hotel in puerto madero is Restaurant Costa Azul.
------------
Retrieving Hotel for City: new norfolk
Closest hotel in new norfolk is The Shingles Riverside Cottages.
------------
Retrieving Hotel for City: sao filipe
Closest hotel in sao filipe is Tortuga B&B.
------------
Retrieving Hotel for City: pochutla
Closest hotel in pochutla is Hotel Posada San Jose.
------------
Retrieving Hotel for City: faya
Closest hotel in faya is صراف الإنماء.
------------
Retrieving Hotel for City: boa vista
Closest hotel in boa vista is Aipana Plaza Hotel.
------------


In [112]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
89,richards bay,ZA,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
156,vila velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
157,puerto madero,MX,14.7208,-92.4215,Restaurant Costa Azul
352,new norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
371,sao filipe,CV,14.8961,-24.4956,Tortuga B&B
378,pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose
387,faya,SA,18.3851,42.4509,صراف الإنماء
434,boa vista,BR,2.8197,-60.6733,Aipana Plaza Hotel


In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [117]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='960px'))